# Week 3
This week I:
- Altered the scaling on my get_data function so that each value has it's own individual scaler, rather than just using one MinMaxScaler object. I then added these scalers to the output dictionary so that I could unscale them later.
- Added a function to create a LSTM model.
- Added a function to predcit the future price based on the number of days in advance that I wanted to predict (in this case 7 days).
- Added a function to create a finished dataframe with the features from the original dataframe, as well as the predicted adjusted close values, and the actual adjusted close values. 
- Added a function to create a graph of the days vs. price (both the actual adjusted close, and predicted adjusted close)
- I then utilised these function on a few different tickers, outputing the predicted price, graph of the dates vs. price, loss, and mean absolute error for each one. Based on these outputted values, I made my buy/sell decisions for this week.   
 

An error occured in my code when I tried to run it again today, which did not occur in earlier runs. The result of my erlier runs were as follows: 

For AMZN:
Future price after 7 days is 3525.85$
Loss: 0.00044708841596730053
Mean absolute error: 82.99083449228097
![AMZN Graph](amazon.png)

For AAPL:
Future price after 7 days is 142.79$
Loss: 0.0009056889684870839
Mean absolute error: 4.301958548367483
![AAPL Graph](apple.png)

For BLK:
Future price after 7 days is 917.52$
Loss: 0.0003547408850863576
Mean absolute error: 27.431870308970947
![BLK Graph](BLK.png)




In [4]:
pip install yahoo_fin

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from yahoo_fin import stock_info as si

In [10]:
si.tickers_nasdaq()

['AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AADR',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AATC',
 'AAWW',
 'AAXJ',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABGI',
 'ABIO',
 'ABMD',
 'ABNB',
 'ABOS',
 'ABSI',
 'ABST',
 'ABTX',
 'ABUS',
 'ABVC',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACBI',
 'ACCD',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACHL',
 'ACHV',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACMR',
 'ACNB',
 'ACOR',
 'ACQR',
 'ACQRU',
 'ACQRW',
 'ACRS',
 'ACRX',
 'ACST',
 'ACT',
 'ACTD',
 'ACTDU',
 'ACTDW',
 'ACTG',
 'ACVA',
 'ACWI',
 'ACWX',
 'ACXP',
 'ADAG',
 'ADALU',
 'ADAP',
 'ADBE',
 'ADER',
 'ADERU',
 'ADERW',
 'ADES',
 'ADGI',
 'ADI',
 'ADIL',
 'ADILW',
 'ADMA',
 'ADMP',
 'ADN',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADRE',
 'ADSK',
 'ADTN',
 'ADTX',
 'ADUS',
 'ADV',
 'ADVM',
 'ADVWW',
 'ADXN',
 'ADXS',
 'AEAC',
 'AEACU',
 'AEACW',
 'AEAEU',
 'AEHA'

In [4]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from collections import deque 

In [6]:
features = ['adjclose', 'volume', 'open', 'high', 'low']

In [19]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [195]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


In [336]:
def get_data(ticker, window=50, predict=1):
    df = si.get_data(ticker)
    output = {}
    output['original df'] = df.copy()

    for col in features: 
        if col not in features:
            return f'Missing {col}'
    
    if 'date' not in df.columns:
        df['date'] = df.index

    scalers = {}
    for feature in features:
        scaler = MinMaxScaler()
        df[feature] = scaler.fit_transform(np.expand_dims(df[feature].values, axis=1))
        scalers[feature] = scaler
    output['scalers'] = scalers
    
    df['predicted'] = df['adjclose'].shift(-predict)
    last_values = np.array(df[features].tail(predict)) 
    df.dropna(inplace=True)
 
    predicted_val = df['predicted'].values
    features_dates = df[features + ['date']].values
    d = deque(maxlen=window)
    window_seq = []

    for i in range(len(features_dates)):
        d.append(features_dates[i])
        if len(d) == window:
            window_seq.append([np.array(d), predicted_val[i]])
    
    # final feature values left after broken up into window sequences, and last values 
    # len(last_values) = (window + predict)
    last_values = list([seq[:len(features)] for seq in d]) + list(last_values) 
    last_values = np.array(last_values).astype(np.float32)
    output['last values'] = last_values

    x, y = [], []
    for X, Y in window_seq:
        x.append(X)
        y.append(Y)

    x = np.array(x)
    y = np.array(y)

    output['x train'], output['x test'], output['y train'], output['y test'] = train_test_split(x, y, test_size=.2)

    dates = output['x test'][:, -1, -1]
    output['test df'] = output['original df'].loc[dates]
    # print(type(output['test df']))
    output['test df'] = output['test df'][~output['test df'].index.duplicated()]
    output['x train'] = output['x train'][:, :, : len(features)].astype(np.float32)
    output['x test'] = output['x test'][:, :, : len(features)].astype(np.float32)

    print(type(output['test df']))
    
    return output
    

In [334]:
get_data('AAPL')

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


{'original df':                   open        high         low       close    adjclose  \
 1980-12-12    0.128348    0.128906    0.128348    0.128348    0.100453   
 1980-12-15    0.122210    0.122210    0.121652    0.121652    0.095213   
 1980-12-16    0.113281    0.113281    0.112723    0.112723    0.088224   
 1980-12-17    0.115513    0.116071    0.115513    0.115513    0.090408   
 1980-12-18    0.118862    0.119420    0.118862    0.118862    0.093029   
 ...                ...         ...         ...         ...         ...   
 2021-11-29  159.369995  161.190002  158.789993  160.240005  160.240005   
 2021-11-30  159.990005  165.520004  159.919998  165.300003  165.300003   
 2021-12-01  167.479996  170.300003  164.529999  164.770004  164.770004   
 2021-12-02  158.740005  164.199997  157.800003  163.759995  163.759995   
 2021-12-03  164.020004  164.960007  159.720001  161.839996  161.839996   
 
                volume ticker  
 1980-12-12  469033600   AAPL  
 1980-12-15  175884

In [209]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import pandas as pd

In [274]:
def make_LSTM_model(window_seq_length, num_features, num_layers=4, units=45, dropout=0.2):
    rnn = Sequential()
    for layer in range(num_layers):
        if layer == 0:
            rnn.add(LSTM(units, return_sequences=True, input_shape=(window_seq_length, num_features)))
        elif layer == num_layers -1:
            rnn.add(LSTM(units))
        else:
            rnn.add(LSTM(units, return_sequences=True))
        rnn.add(Dropout(dropout))
    rnn.add(Dense(units=1))
    rnn.compile(optimizer='adam', metrics='mean_absolute_error', loss='huber_loss')
    return rnn

In [275]:
import os
import time

In [364]:
ticker = 'AMZN'
window = 50
predict = 7
layers = 4
units = 256
dropout = 0.2
date = time.strftime("%Y-%m-%d")
eps = 10
b_size=64

ticker_filename = os.path.join('data', f'{date}_{ticker}.csv')
model_name = f'{date}_{ticker}'

data = get_data(ticker, window, predict)
data['original df'].to_csv(ticker_filename)

model = make_LSTM_model(window, len(features), layers, units, dropout)
m_checkpoint = ModelCheckpoint(filepath=os.path.join('model_checkpoint', model_name + '.h5'), 
                                save_best_only=True, save_weights_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join('tensorboard', model_name))

model_history = model.fit(data['x train'], data['y train'], batch_size=b_size, epochs=eps, validation_data=(data['x test'], data['y test']), 
                            callbacks=[m_checkpoint, tensorboard], verbose=1)
                            


<class 'pandas.core.frame.DataFrame'>
Epoch 1/10
77/77 [==============================] - ETA: 0s - loss: 0.0048 - mean_absolute_error: 0.0392
Epoch 00001: val_loss improved from inf to 0.00015, saving model to model_checkpoint/2021-12-03_AMZN.h5
77/77 [==============================] - 67s 781ms/step - loss: 0.0048 - mean_absolute_error: 0.0392 - val_loss: 1.4674e-04 - val_mean_absolute_error: 0.0083
Epoch 2/10
77/77 [==============================] - ETA: 0s - loss: 3.3711e-04 - mean_absolute_error: 0.0122
Epoch 00002: val_loss did not improve from 0.00015
77/77 [==============================] - 39s 510ms/step - loss: 3.3711e-04 - mean_absolute_error: 0.0122 - val_loss: 2.1652e-04 - val_mean_absolute_error: 0.0089
Epoch 3/10
77/77 [==============================] - ETA: 0s - loss: 3.8822e-04 - mean_absolute_error: 0.0131
Epoch 00003: val_loss improved from 0.00015 to 0.00014, saving model to model_checkpoint/2021-12-03_AMZN.h5
77/77 [==============================] - 39s 513ms/step 

In [363]:
os.mkdir('tensorboard')
os.mkdir('model_checkpoint')
os.mkdir('data')

In [224]:
import matplotlib.pyplot as plt

In [365]:
def make_prediction(m, d):
    last_values = d['last values'][-predict:]
    last_values = np.expand_dims(last_values, 0)
    prediction = m.predict(last_values)
    predicted_price = d['scalers']['adjclose'].inverse_transform(prediction)[0][0]
    return predicted_price

In [366]:
def get_finished_df(m, d):
    x_test = d['x test']
    y_test = d['y test']

    y_prediction = m.predict(x_test)
    y_test = np.squeeze(d['scalers']['adjclose'].inverse_transform(np.expand_dims(y_test, axis=0)))
    y_prediction = np.squeeze(d['scalers']['adjclose'].inverse_transform(y_prediction))
    
    test_df = d['test df']

    test_df[f'actual adjclose {predict}'] = y_test
    test_df[f'adjclose {predict}'] = y_prediction
    test_df.sort_index(inplace=True)
    finished_df = test_df
    
    return finished_df


In [367]:
def plot(df):
    plt.plot(df[f'actual adjclose {predict}'], c='b')
    plt.plot(df[f'adjclose {predict}'], c='r')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.legend(['Actual Price', 'Predicted Price'])
    plt.show()

In [368]:
model_path = os.path.join('model_checkpoint', model_name) + '.h5' 
model.load_weights(model_path)


In [369]:
loss, mae = model.evaluate(data['x test'], data['y test'], verbose=1)
mean_abs_error = data['scalers']['adjclose'].inverse_transform([[mae]])[0][0]

39/39 [==============================] - 4s 99ms/step - loss: 1.1870e-04 - mean_absolute_error: 0.0072


In [328]:
print(type(data['test df']))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [370]:
finished_df = get_finished_df(model, data)
future_price = make_prediction(model, data)
print(f'Future price after {predict} days is {future_price:.2f}$')
print(f'Loss: {loss}')
print(f'Mean absolute error: {mean_abs_error}')
plot(finished_df)

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [355]:
csv_result_folder = 'csv_results'
if not os.path.isdir(csv_result_folder):
    os.mkdir(csv_result_folder)
csv_filename = os.path.join(csv_result_folder, model_name + '.csv')
finished_df.to_csv(csv_filename)


In [360]:
ticker = 'AAPL'
window = 50
predict = 7
layers = 4
units = 256
dropout = 0.2
date = time.strftime("%Y-%m-%d")
eps = 10
b_size=64

ticker_filename = os.path.join('data', f'{date}_{ticker}.csv')
model_name = f'{date}_{ticker}'

data = get_data(ticker, window, predict)
data['original df'].to_csv(ticker_filename)

model = make_LSTM_model(window, len(features), layers, units, dropout)
m_checkpoint = ModelCheckpoint(filepath=os.path.join('model_checkpoint', model_name + '.h5'), 
                                save_best_only=True, save_weights_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join('tensorboard', model_name))

model_history = model.fit(data['x train'], data['y train'], batch_size=b_size, epochs=eps, validation_data=(data['x test'], data['y test']), 
                            callbacks=[m_checkpoint, tensorboard], verbose=1)

<class 'pandas.core.frame.DataFrame'>
Epoch 1/50
129/129 [==============================] - ETA: 0s - loss: 0.0015 - mean_absolute_error: 0.0200
Epoch 00001: val_loss improved from inf to 0.00007, saving model to model_checkpoint/2021-12-03_AAPL.h5
129/129 [==============================] - 78s 543ms/step - loss: 0.0015 - mean_absolute_error: 0.0200 - val_loss: 6.7249e-05 - val_mean_absolute_error: 0.0048
Epoch 2/50
129/129 [==============================] - ETA: 0s - loss: 1.5471e-04 - mean_absolute_error: 0.0077
Epoch 00002: val_loss did not improve from 0.00007
129/129 [==============================] - 67s 516ms/step - loss: 1.5471e-04 - mean_absolute_error: 0.0077 - val_loss: 1.2943e-04 - val_mean_absolute_error: 0.0072
Epoch 3/50
129/129 [==============================] - ETA: 0s - loss: 2.0889e-04 - mean_absolute_error: 0.0087
Epoch 00003: val_loss improved from 0.00007 to 0.00006, saving model to model_checkpoint/2021-12-03_AAPL.h5
129/129 [==============================] - 66s

In [361]:
model_path = os.path.join('model_checkpoint', model_name) + '.h5' 
model.load_weights(model_path)
loss, mae = model.evaluate(data['x test'], data['y test'], verbose=1)
mean_abs_error = data['scalers']['adjclose'].inverse_transform([[mae]])[0][0]
finished_df = get_finished_df(model, data)
future_price = make_prediction(model, data)
print(f'Future price after {predict} days is {future_price:.2f}$')
print(f'Loss: {loss}')
print(f'Mean absolute error: {mean_abs_error}')
plot(finished_df)

csv_result_folder = 'csv_results'
if not os.path.isdir(csv_result_folder):
    os.mkdir(csv_result_folder)
csv_filename = os.path.join(csv_result_folder, model_name + '.csv')
finished_df.to_csv(csv_filename)


65/65 [==============================] - 5s 76ms/step - loss: 3.2125e-05 - mean_absolute_error: 0.0037


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [371]:
ticker = 'BLK'
window = 50
predict = 7
layers = 4
units = 256
dropout = 0.2
date = time.strftime("%Y-%m-%d")
eps = 10
b_size=64

ticker_filename = os.path.join('data', f'{date}_{ticker}.csv')
model_name = f'{date}_{ticker}'

data = get_data(ticker, window, predict)
data['original df'].to_csv(ticker_filename)

model = make_LSTM_model(window, len(features), layers, units, dropout)
m_checkpoint = ModelCheckpoint(filepath=os.path.join('model_checkpoint', model_name + '.h5'), 
                                save_best_only=True, save_weights_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join('tensorboard', model_name))

model_history = model.fit(data['x train'], data['y train'], batch_size=b_size, epochs=eps, validation_data=(data['x test'], data['y test']), 
                            callbacks=[m_checkpoint, tensorboard], verbose=1)

<class 'pandas.core.frame.DataFrame'>
Epoch 1/10
70/70 [==============================] - ETA: 0s - loss: 0.0040 - mean_absolute_error: 0.0441
Epoch 00001: val_loss improved from inf to 0.00050, saving model to model_checkpoint/2021-12-03_BLK.h5
70/70 [==============================] - 52s 587ms/step - loss: 0.0040 - mean_absolute_error: 0.0441 - val_loss: 5.0194e-04 - val_mean_absolute_error: 0.0225
Epoch 2/10
70/70 [==============================] - ETA: 0s - loss: 3.8548e-04 - mean_absolute_error: 0.0172
Epoch 00002: val_loss improved from 0.00050 to 0.00022, saving model to model_checkpoint/2021-12-03_BLK.h5
70/70 [==============================] - 39s 559ms/step - loss: 3.8548e-04 - mean_absolute_error: 0.0172 - val_loss: 2.2256e-04 - val_mean_absolute_error: 0.0134
Epoch 3/10
70/70 [==============================] - ETA: 0s - loss: 4.9000e-04 - mean_absolute_error: 0.0187
Epoch 00003: val_loss did not improve from 0.00022
70/70 [==============================] - 35s 507ms/step - 

In [372]:
model_path = os.path.join('model_checkpoint', model_name) + '.h5' 
model.load_weights(model_path)
loss, mae = model.evaluate(data['x test'], data['y test'], verbose=1)
mean_abs_error = data['scalers']['adjclose'].inverse_transform([[mae]])[0][0]
finished_df = get_finished_df(model, data)
future_price = make_prediction(model, data)
print(f'Future price after {predict} days is {future_price:.2f}$')
print(f'Loss: {loss}')
print(f'Mean absolute error: {mean_abs_error}')
plot(finished_df)

csv_result_folder = 'csv_results'
if not os.path.isdir(csv_result_folder):
    os.mkdir(csv_result_folder)
csv_filename = os.path.join(csv_result_folder, model_name + '.csv')
finished_df.to_csv(csv_filename)

35/35 [==============================] - 3s 72ms/step - loss: 1.7786e-04 - mean_absolute_error: 0.0116


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fro